In [20]:
import torchtext
import pandas as pd
from torchtext.data.utils import get_tokenizer
from torchtext import datasets
import time
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np

In [3]:
torchtext.utils.download_from_url(datasets.text_classification.URLS['YelpReviewPolarity'])
!tar -C .data -xvf .data/yelp_review_polarity_csv.tar.gz

x yelp_review_polarity_csv/
x yelp_review_polarity_csv/readme.txt
x yelp_review_polarity_csv/test.csv
x yelp_review_polarity_csv/train.csv


In [4]:
train_df = pd.read_csv('.data/yelp_review_polarity_csv/train.csv',names=['label','text'])
test_df = pd.read_csv('.data/yelp_review_polarity_csv/test.csv',names=['label','text'])

In [7]:
# preprocess text with torchtext
tokenize=get_tokenizer('basic_english')
train_df['text'] = train_df['text'].apply(lambda text: ' '.join(tokenize(text)))
test_df['text'] = test_df['text'].apply(lambda text: ' '.join(tokenize(text)))

In [12]:
count_vect = CountVectorizer()
trainCounts = count_vect.fit_transform(train_df['text'])

In [15]:
tfidf_transformer = TfidfTransformer()
trainTfidf = tfidf_transformer.fit_transform(trainCounts)
trainTfidf.shape

(560000, 224898)

In [22]:
start_time = time.time()
classifier = (LogisticRegression(n_jobs=-1,solver='sag',max_iter=10,multi_class='multinomial',verbose=2)
              .fit(trainTfidf, train_df['label']))
secs = time.time() - start_time
secs

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


max_iter reached after 15 seconds


/Users/yonatang/nlp_techniques/venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   15.7s finished


15.953402996063232

In [23]:
testCounts = count_vect.transform(test_df['text'])
testTfidf = tfidf_transformer.transform(testCounts)
predicted = classifier.predict(testTfidf)
np.mean(predicted == test_df['label'])

0.9397368421052632